In [ ]:
# Cell 1: Import libraries + path + data reading & merging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from joblib import Parallel, delayed
import math
import os
import time
from scipy.stats import gaussian_kde, probplot, shapiro
import statsmodels.api as sm

start_time = time.time()

# ==============================================================
# 1. Paths & site info
# ==============================================================
site_info = [
    (r"E:\jupyter_data\Hapke\Submit\data\Original\ave_01_o.csv",
     r"E:\jupyter_data\Hapke\Submit\data\Original\shice_01.csv",
     r"E:\jupyter_data\Hapke\Submit\data\processed\Hapke_result",
     30.57, 0.0, 153.28, 59.31, 0.0, 261.17, "01"),
    (r"E:\jupyter_data\Hapke\Submit\data\Original\ave_03_o.csv",
     r"E:\jupyter_data\Hapke\Submit\data\Original\shice_03.csv",
     r"E:\jupyter_data\Hapke\Submit\data\processed\Hapke_result",
     37.42, 0.0, 130.25, 43.69, 0.0, 241.63, "03")
]

output_root = site_info[0][2]
os.makedirs(output_root, exist_ok=True)

# ==============================================================
# 2. Plot settings
# ==============================================================
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']
plt.rcParams['font.size'] = 10
plt.rcParams['axes.unicode_minus'] = False

# ==============================================================
# 3. Hapke model functions
# ==============================================================
def cos_g(i, e, phi): return np.cos(i) * np.cos(e) + np.sin(i) * np.sin(e) * np.cos(phi)
def mu_0(i): return np.cos(i)
def mu(e): return np.cos(e)

def B_g(g, B_0, h):
    tan_half_g = np.tan(g / 2)
    return B_0 / (1 + (1 / h) * tan_half_g if tan_half_g != 0 else 1 + 1e-10)

def P_g(g, b, c):
    cos_g_val = np.cos(g)
    d1 = max(1 - 2 * b * cos_g_val + b**2, 1e-10)
    d2 = max(1 + 2 * b * cos_g_val + b**2, 1e-10)
    b = min(b, 0.99999)
    term1 = ((1 + c) / 2) * (1 - b**2) / (d1)**1.5
    term2 = ((1 - c) / 2) * (1 - b**2) / (d2)**1.5
    return term1 + term2

def H(mu, w): return (1 + 2 * mu) / (1 + 2 * mu * np.sqrt(1 - w))

def R_hapke(mu_0, mu, g, w, B_0, h, b, c):
    B = B_g(g, B_0, h)
    P = P_g(g, b, c)
    H0 = H(mu_0, w)
    H_mu = H(mu, w)
    return (w / (4 * np.pi * (mu_0 + mu))) * ((1 + B) * P + H0 * H_mu - 1)

# ==============================================================
# 4. Wavelength band definitions
# ==============================================================
wavelengths = [350 + (i-1)*4 for i in range(1, 165)]
selected_wavelengths = wavelengths[10:154]
band_range = (11, 154)
n_bands = band_range[1] - band_range[0] + 1

# ==============================================================
# 5. Read & merge data
# ==============================================================
all_ref_1a, all_ref_1b, all_tree_ids, all_geom = [], [], [], []

for idx, (inp_file, shice_file, _, i1_deg, e1_deg, phi1_deg,
          i2_deg, e2_deg, phi2_deg, prefix) in enumerate(site_info):
    shice_df = pd.read_csv(shice_file)
    valid_ids = shice_df['Tree_ID'].values
    df = pd.read_csv(inp_file)
    df = df[df['Tree_ID'].isin(valid_ids)].copy()
    df['Tree_ID'] = prefix + '_' + df['Tree_ID'].astype(str)

    start_band, end_band = band_range
    bands_1a = [f'1a_{350 + (i-1)*4}nm' for i in range(start_band, end_band + 1)]
    bands_1b = [f'1b_{350 + (i-1)*4}nm' for i in range(start_band, end_band + 1)]

    ref_1a = df[bands_1a].values
    ref_1b = df[bands_1b].values
    tree_ids = df['Tree_ID'].values

    all_ref_1a.append(ref_1a)
    all_ref_1b.append(ref_1b)
    all_tree_ids.extend(tree_ids)

    i1, e1, phi1 = map(math.radians, [i1_deg, e1_deg, phi1_deg])
    i2, e2, phi2 = map(math.radians, [i2_deg, e2_deg, phi2_deg])
    mu0_1 = mu_0(i1); mu_1 = mu(e1); g1 = np.arccos(cos_g(i1, e1, phi1))
    mu0_2 = mu_0(i2); mu_2 = mu(e2); g2 = np.arccos(cos_g(i2, e2, phi2))
    geom = (mu0_1, mu_1, g1, mu0_2, mu_2, g2)
    all_geom.extend([geom] * len(tree_ids))

actual_reflectance_1a = np.vstack(all_ref_1a)
actual_reflectance_1b = np.vstack(all_ref_1b)
tree_ids = np.array(all_tree_ids)
n_samples = actual_reflectance_1a.shape[0]

# Reindex tree_ids
new_tree_id = np.arange(1, n_samples + 1)
tree_id_map = pd.DataFrame({'Original_Tree_ID': tree_ids, 'New_Tree_ID': new_tree_id})
tree_id_map.to_csv(os.path.join(output_root, "Tree_ID_mapping.csv"), index=False)

print(f"Data loaded. Total samples: {n_samples}")

In [ ]:
# Cell 2: Optimization fitting + save parameters & fitted values
def optimize_sample_with_geom(sample_idx, actual_1a, actual_1b, geom_param):
    mu0_1, mu_1, g1, mu0_2, mu_2, g2 = geom_param
    def objective(params):
        w_vals = np.clip(params[:n_bands], 0, 1)
        B0 = np.clip(params[n_bands], 0, 1)
        h = np.clip(params[n_bands+1], 0.01, 1)
        b = np.clip(params[n_bands+2], 0, 1)
        c = np.clip(params[n_bands+3], -1, 1)
        model_1a = R_hapke(mu0_1, mu_1, g1, w_vals, B0, h, b, c)
        model_1b = R_hapke(mu0_2, mu_2, g2, w_vals, B0, h, b, c)
        return np.sum((model_1a - actual_1a)**2) + np.sum((model_1b - actual_1b)**2)
    init = np.concatenate([np.full(n_bands, 0.5), [1.0, 0.1, 0.5, 0.0]])
    bounds = [(0,1)]*n_bands + [(0,1), (0.01,1), (0,1), (-1,1)]
    res = minimize(objective, init, bounds=bounds, method='L-BFGS-B', options={'maxfun': 1_000_000, 'maxiter': 1_000_000})
    if not res.success:
        print(f"Warning: Sample {sample_idx} optimization failed: {res.message}")
    w = res.x[:n_bands]
    B0, h, b, c = res.x[n_bands:n_bands+4]
    fit_1a = R_hapke(mu0_1, mu_1, g1, w, B0, h, b, c)
    fit_1b = R_hapke(mu0_2, mu_2, g2, w, B0, h, b, c)
    return w, B0, h, b, c, fit_1a, fit_1b

print(f"Starting parallel fitting ({n_samples} samples)...")
results = Parallel(n_jobs=-1, verbose=10)(
    delayed(optimize_sample_with_geom)(i, actual_reflectance_1a[i], actual_reflectance_1b[i], all_geom[i])
    for i in range(n_samples)
)
w_est, B0_est, h_est, b_est, c_est, fit_1a_list, fit_1b_list = zip(*results)
w_est = list(w_est); B0_est = list(B0_est); h_est = list(h_est)
b_est = list(b_est); c_est = list(c_est)
fit_1a_list = list(fit_1a_list); fit_1b_list = list(fit_1b_list)

# Save parameters
w_all = np.column_stack(w_est).T
w_cols = {f'w_{wl}nm': w_all[:, i] for i, wl in enumerate(selected_wavelengths)}
params_df = pd.DataFrame({'New_Tree_ID': new_tree_id, 'Original_Tree_ID': tree_ids,
                          'B_0': B0_est, 'h': h_est, 'b': b_est, 'c': c_est})
params_df = pd.concat([params_df, pd.DataFrame(w_cols)], axis=1)
params_df.to_csv(os.path.join(output_root, "hapke_parameters_merged.csv"), index=False)

# Save fitted reflectance
fitted_cols = [pd.Series(new_tree_id, name='New_Tree_ID'), pd.Series(tree_ids, name='Original_Tree_ID')]
for i, wl in enumerate(selected_wavelengths):
    col_1a = [fit_1a_list[j][i] for j in range(n_samples)]
    col_1b = [fit_1b_list[j][i] for j in range(n_samples)]
    fitted_cols.append(pd.Series(col_1a, name=f'Fitted_1a_{wl}nm'))
    fitted_cols.append(pd.Series(col_1b, name=f'Fitted_1b_{wl}nm'))
pd.concat(fitted_cols, axis=1).to_csv(os.path.join(output_root, "merged_fitted_reflectance_390-962.csv"), index=False)

print("Fitting completed, parameters have been saved.")

In [ ]:
# Cell 3: Diagnostic Plots (can be run independently)
n_samples_to_plot = n_samples          

# --------------------------------------------------------------
# Accuracy Metrics (unchanged)
# --------------------------------------------------------------
def calc_metrics(fit, act):
    ss_tot = np.sum((act - act.mean())**2)
    ss_res = np.sum((act - fit)**2)
    r2 = 1 - ss_res/ss_tot if ss_tot else 0
    rmse = np.sqrt(ss_res/len(act))
    return r2, rmse

all_act = np.concatenate([np.concatenate(actual_reflectance_1a),
                          np.concatenate(actual_reflectance_1b)])
all_fit = np.concatenate([np.concatenate(fit_1a_list),
                          np.concatenate(fit_1b_list)])
total_r2, total_rmse = calc_metrics(all_fit, all_act)

sample_metrics = [
    (new_tree_id[i], tree_ids[i],
     *calc_metrics(np.concatenate([fit_1a_list[i], fit_1b_list[i]]),
                   np.concatenate([actual_reflectance_1a[i], actual_reflectance_1b[i]])))
    for i in range(n_samples)
]

# NEW: Statistics of R2 distribution for each sample
r2_values = [metric[2] for metric in sample_metrics]  # Extract R2 for all samples
r2_array = np.array(r2_values)

# Calculate statistics
r2_mean = np.mean(r2_array)
r2_median = np.median(r2_array)
r2_std = np.std(r2_array)
r2_min = np.min(r2_array)
r2_max = np.max(r2_array)
r2_q1 = np.percentile(r2_array, 25)
r2_q3 = np.percentile(r2_array, 75)
r2_iqr = r2_q3 - r2_q1

# Print statistics
print("R2 distribution statistics for each sample:")
print(f"Mean: {r2_mean:.4f}")
print(f"Median: {r2_median:.4f}")
print(f"Std Dev: {r2_std:.4f}")
print(f"Min: {r2_min:.4f}")
print(f"Max: {r2_max:.4f}")
print(f"Q1 (25% percentile): {r2_q1:.4f}")
print(f"Q3 (75% percentile): {r2_q3:.4f}")
print(f"IQR (Q3 - Q1): {r2_iqr:.4f}")

# Optional: Save statistics to file (if needed)
stats_file = os.path.join(output_root, 'r2_distribution_stats.txt')
with open(stats_file, 'w') as f:
    f.write("R2 distribution statistics for each sample:\n")
    f.write(f"Mean: {r2_mean:.4f}\n")
    f.write(f"Median: {r2_median:.4f}\n")
    f.write(f"Std Dev: {r2_std:.4f}\n")
    f.write(f"Min: {r2_min:.4f}\n")
    f.write(f"Max: {r2_max:.4f}\n")
    f.write(f"Q1 (25% percentile): {r2_q1:.4f}\n")
    f.write(f"Q3 (75% percentile): {r2_q3:.4f}\n")
    f.write(f"IQR (Q3 - Q1): {r2_iqr:.4f}\n")
print(f"R2 statistics have been saved to {stats_file}")

# --------------------------------------------------------------
# Plotting (uniform style, no grid, no title)
# --------------------------------------------------------------
all_fitted_vals, all_residuals = [], []

# ---------- 1. Overall scatter plot (density colored) ----------
plt.figure(figsize=(5, 4))
xy = np.vstack([all_act, all_fit])
density = gaussian_kde(xy)(xy)
density = (density - density.min()) / (density.max() - density.min())
plt.scatter(all_act, all_fit, c=density, cmap='viridis_r', s=1, alpha=1)
plt.colorbar(label='Point Density')
plt.plot([0, 1], [0, 1], 'r--', label='1:1 Line')
plt.xlabel('Actual Reflectance')
plt.ylabel('Fitted Reflectance')
plt.legend(loc='lower right')
plt.text(0.05, 0.93,
         f'$R^2$ = {total_r2:.4f}\nRMSE = {total_rmse:.4f}',
         transform=plt.gca().transAxes, fontsize=10,
         verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
plt.xlim(0, max(all_act.max(), all_fit.max()) * 1.1)
plt.ylim(0, max(all_act.max(), all_fit.max()) * 1.1)
plt.grid(False)   # Grid off
plt.savefig(os.path.join(output_root,
                         'scatter_actual_vs_fitted_merged_samples.png'),
            dpi=600, bbox_inches='tight')
plt.close()

# ---------- 2. Loop over individual samples ----------
for i in range(min(n_samples, n_samples_to_plot)):
    new_id = new_tree_id[i]
    sample_act = np.concatenate([actual_reflectance_1a[i],
                                 actual_reflectance_1b[i]])
    sample_fit = np.concatenate([fit_1a_list[i],
                                 fit_1b_list[i]])
    residuals = sample_act - sample_fit
    all_fitted_vals.extend(sample_fit)
    all_residuals.extend(residuals)

    # ---- Individual scatter plot ----
    plt.figure(figsize=(5, 4))
    plt.scatter(sample_act, sample_fit, c='blue', s=1)
    plt.plot([0, 1], [0, 1], 'r--', label='1:1 Line')
    plt.xlabel('Actual Reflectance')
    plt.ylabel('Fitted Reflectance')
    plt.legend(loc='lower right')
    r2, rmse = sample_metrics[i][2], sample_metrics[i][3]
    plt.text(0.05, 0.93,
             f'$R^2$ = {r2:.4f}\nRMSE = {rmse:.4f}',
             transform=plt.gca().transAxes, fontsize=10,
             verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    plt.xlim(0, max(sample_act.max(), sample_fit.max()) * 1.1)
    plt.ylim(0, max(sample_act.max(), sample_fit.max()) * 1.1)
    plt.grid(False)
    plt.savefig(os.path.join(output_root,
                             f'scatter_Tree_ID_{new_id}_merged.png'),
                dpi=600, bbox_inches='tight')
    plt.close()

    # ---- Residuals vs. Fitted (individual) ----
    plt.figure(figsize=(5, 4))
    plt.scatter(sample_fit, residuals, c='blue', s=15,
                alpha=0.8, edgecolors='none')
    if len(sample_fit) > 5:
        smooth = sm.nonparametric.lowess(residuals, sample_fit, frac=0.3)
        plt.plot(smooth[:, 0], smooth[:, 1], 'r-', lw=1.5,
                 label='LOESS Trend')
    plt.axhline(0, color='black', linestyle='--', linewidth=0.8)
    plt.xlabel('Fitted Values')
    plt.ylabel('Residuals')
    plt.legend(loc='lower right')
    plt.text(0.05, 0.93,
             f'$R^2$ = {r2:.4f}\nRMSE = {rmse:.4f}',
             transform=plt.gca().transAxes, fontsize=10,
             verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    plt.grid(False)
    plt.savefig(os.path.join(output_root,
                             f'residual_vs_fitted_Tree_ID_{new_id}.png'),
                dpi=600, bbox_inches='tight')
    plt.close()

    # ---- QQ plot (individual) ----
    plt.figure(figsize=(5, 4))
    probplot(residuals, dist="norm", plot=plt)
    lines = plt.gca().get_lines()
    lines[0].set_color('blue')   # Points
    lines[0].set_markersize(2)
    lines[1].set_color('red')    # Reference line
    plt.xlabel('Theoretical Quantiles')
    plt.ylabel('Sample Quantiles')
    plt.gca().set_title('')      # Ensure no title
    plt.grid(False)
    plt.savefig(os.path.join(output_root,
                             f'qq_plot_New_Tree_ID_{new_id}.png'),
                dpi=600, bbox_inches='tight')
    plt.close()

    # ---- Residual histogram (individual) ----
    plt.figure(figsize=(5, 4))
    plt.hist(residuals, bins=20, color='blue',
             alpha=0.7, edgecolor='black', linewidth=0.5)
    plt.xlabel('Residuals')
    plt.ylabel('Frequency')
    plt.grid(False)
    plt.savefig(os.path.join(output_root,
                             f'residual_histogram_New_Tree_ID_{new_id}.png'),
                dpi=600, bbox_inches='tight')
    plt.close()

# ---------- 3. Overall Residuals vs. Fitted (density) ----------
all_fitted_vals = np.array(all_fitted_vals)
all_residuals   = np.array(all_residuals)
xy = np.vstack([all_fitted_vals, all_residuals])
density = gaussian_kde(xy)(xy)
density = (density - density.min()) / (density.max() - density.min() + 1e-8)

plt.figure(figsize=(6, 4.5))
scatter = plt.scatter(all_fitted_vals, all_residuals,
                      c=density, cmap='viridis_r', s=2, alpha=0.7)
plt.colorbar(scatter, label='Point Density')
if len(all_fitted_vals) > 10:
    smooth = sm.nonparametric.lowess(all_residuals, all_fitted_vals, frac=0.2)
    plt.plot(smooth[:, 0], smooth[:, 1], 'r-', lw=2,
             label='LOESS Trend')
plt.axhline(0, color='black', linestyle='--', linewidth=0.8)
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.legend(loc='lower right')
plt.text(0.05, 0.93,
         f'$R^2$ = {total_r2:.4f}\nRMSE = {total_rmse:.4f}',
         transform=plt.gca().transAxes, fontsize=10,
         verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
plt.grid(False)
plt.savefig(os.path.join(output_root,
                         'overall_residual_vs_fitted_density.png'),
            dpi=600, bbox_inches='tight')
plt.close()

# ---------- 4. Overall QQ plot ----------
plt.figure(figsize=(5, 4))
probplot(all_residuals, dist="norm", plot=plt)
lines = plt.gca().get_lines()
lines[0].set_color('green')   # Points
lines[1].set_color('red')     # Reference line
lines[0].set_markersize(2)
plt.xlabel('Theoretical Quantiles')
plt.ylabel('Sample Quantiles')
plt.gca().set_title('')
plt.grid(False)
plt.savefig(os.path.join(output_root, 'overall_qq_plot.png'),
            dpi=600, bbox_inches='tight')
plt.close()

# ---------- 5. Overall residual histogram ----------
plt.figure(figsize=(5, 4))
plt.hist(all_residuals, bins=50, color='green',
         alpha=0.7, edgecolor='black', linewidth=0.5)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.grid(False)
plt.savefig(os.path.join(output_root, 'overall_residual_histogram.png'),
            dpi=600, bbox_inches='tight')
plt.close()

print(f"Diagnostic plots saved (first {n_samples_to_plot} samples + overall plots)")
print("Diagnostic plotting complete")

In [ ]:
 # Cell 4: Bootstrap & Overall Uncertainty (No Title)
# ==============================================================
n_bootstraps = 1000

def bootstrap_sample(sample_idx, actual_1a, actual_1b, fitted_1a, fitted_1b, geom_param, new_id):
    residuals_1a = actual_1a - fitted_1a
    residuals_1b = actual_1b - fitted_1b
    
    boot_w = np.zeros((n_bootstraps, n_bands))
    boot_B0 = np.zeros(n_bootstraps)
    boot_h = np.zeros(n_bootstraps)
    boot_b = np.zeros(n_bootstraps)
    boot_c = np.zeros(n_bootstraps)
    
    for boot_idx in range(n_bootstraps):
        boot_res_1a = np.random.choice(residuals_1a, size=len(residuals_1a), replace=True)
        boot_res_1b = np.random.choice(residuals_1b, size=len(residuals_1b), replace=True)
        boot_actual_1a = fitted_1a + boot_res_1a
        boot_actual_1b = fitted_1b + boot_res_1b
        
        w, B0, h, b, c, _, _ = optimize_sample_with_geom(sample_idx, boot_actual_1a, boot_actual_1b, geom_param)
        
        boot_w[boot_idx] = w
        boot_B0[boot_idx] = B0
        boot_h[boot_idx] = h
        boot_b[boot_idx] = b
        boot_c[boot_idx] = c
    
    w_mean = np.mean(boot_w, axis=0)
    w_std = np.std(boot_w, axis=0)
    w_ci_low = np.percentile(boot_w, 2.5, axis=0)
    w_ci_high = np.percentile(boot_w, 97.5, axis=0)
    
    B0_mean = np.mean(boot_B0)
    B0_std = np.std(boot_B0)
    B0_ci = np.percentile(boot_B0, [2.5, 97.5])
    
    h_mean = np.mean(boot_h)
    h_std = np.std(boot_h)
    h_ci = np.percentile(boot_h, [2.5, 97.5])
    
    b_mean = np.mean(boot_b)
    b_std = np.std(boot_b)
    b_ci = np.percentile(boot_b, [2.5, 97.5])
    
    c_mean = np.mean(boot_c)
    c_std = np.std(boot_c)
    c_ci = np.percentile(boot_c, [2.5, 97.5])
    
    # Save CSV
    boot_data = {
        'Wavelength_nm': selected_wavelengths,
        'w_mean': w_mean,
        'w_std': w_std,
        'w_ci_low': w_ci_low,
        'w_ci_high': w_ci_high
    }
    boot_df = pd.DataFrame(boot_data)
    boot_file = os.path.join(output_root, f'bootstrap_w_uncertainty_New_Tree_ID_{new_id}.csv')
    boot_df.to_csv(boot_file, index=False)
    
    # Plot w uncertainty
    plt.figure(figsize=(8, 4))
    plt.plot(selected_wavelengths, w_mean, label='Mean w', color='blue')
    plt.fill_between(selected_wavelengths, w_ci_low, w_ci_high, color='blue', alpha=0.3, label='95% CI')
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('w')
    plt.legend()
    w_uncert_file = os.path.join(output_root, f'w_uncertainty_plot_New_Tree_ID_{new_id}.png')
    plt.savefig(w_uncert_file, dpi=600, bbox_inches='tight')
    plt.close()
    
    return boot_w, boot_B0, boot_h, boot_b, boot_c

# Parallel Bootstrap
print("Starting parallel bootstrap uncertainty analysis...")
boot_results = Parallel(n_jobs=-1, verbose=10)(
    delayed(bootstrap_sample)(i, actual_reflectance_1a[i], actual_reflectance_1b[i], fit_1a_list[i], fit_1b_list[i], all_geom[i], new_tree_id[i]) 
    for i in range(n_samples)
)

# ==============================================================
# 13. Overall Bootstrap Uncertainty (No Title)
# ==============================================================
all_boot_w = np.vstack([res[0] for res in boot_results])

overall_w_mean = np.mean(all_boot_w, axis=0)
overall_w_std = np.std(all_boot_w, axis=0)
overall_w_ci_low = np.percentile(all_boot_w, 2.5, axis=0)
overall_w_ci_high = np.percentile(all_boot_w, 97.5, axis=0)

overall_boot_df = pd.DataFrame({
    'Wavelength_nm': selected_wavelengths,
    'w_mean': overall_w_mean,
    'w_std': overall_w_std,
    'w_ci_low': overall_w_ci_low,
    'w_ci_high': overall_w_ci_high
})
overall_boot_file = os.path.join(output_root, "overall_bootstrap_w_uncertainty.csv")
overall_boot_df.to_csv(overall_boot_file, index=False)
print(f"Overall bootstrap w uncertainty saved → {overall_boot_file}")

plt.figure(figsize=(8, 4))
plt.plot(selected_wavelengths, overall_w_mean, label='Overall Mean w', color='green')
plt.fill_between(selected_wavelengths, overall_w_ci_low, overall_w_ci_high, color='green', alpha=0.3, label='95% CI')
plt.xlabel('Wavelength (nm)')
plt.ylabel('w')
plt.legend()
overall_uncert_file = os.path.join(output_root, "overall_w_uncertainty_plot.png")
plt.savefig(overall_uncert_file, dpi=600, bbox_inches='tight')
plt.close()
print(f"Overall w uncertainty plot saved → {overall_uncert_file}")